In [182]:
import glob
# This file should run basically insta fast. Bad coding for sure 
# There is a place where i sometimes comment out lin/rbf/sigmoid stuff
# Dont do that, it really helps for some subjects. 
# Anything. Leave only linear. For fast testing of best features
# But dont do that either. 

genSavePath = "F:/PythonProjects/NietoExcercise-1/SavedResults/"
day = "10" # Which month and days result you want to visualize in table
month = 11

repetitionValue = 32
repetitionName =   "udrlv-1feat" # "udrliplotnoAda3"  #  "udrlBC4CVTest" # 
#  "udrliplotnoAda3hyperparams" 
repetitionString = f"{repetitionValue}{repetitionName}"
curSavePath = f"{genSavePath}{day}-{month}-{repetitionString}/"
savedResults = glob.glob(curSavePath + "savedBestSeed*")

saveTop = True
if saveTop:
    saveTopName = "top1udrlv"
else:
    saveTopName = None
topAmount = 20
skipSubjects = []
onlySignificant = True
minAcc = 0.1
print(len(savedResults))
print(len(savedResults)/9)

90
10.0


In [183]:
import pandas as pd
from array import ArrayType
from requests import delete
from tabulate import tabulate
from prettytable import PrettyTable as pt
import numpy as np


nameDict = dict()
nrOfNames = 0
kernelDict = dict()
nrOfKernel = 0
nrOfCombination = 0
currSeed = 0
currSubj = 0
combinationList = []
combinationResult = []


results = []
for path in savedResults:
    results.append(np.load(path, allow_pickle=True))

subjectAmount = 10
for x in range(subjectAmount):
    combinationResult.append([])

nrOfSeedsPerSubject = np.zeros(subjectAmount)
for s, subject in enumerate(results):
    currSubj = 0
    currSeed = 0
    subjectBestArray = np.empty([len(subject)], dtype=object)

    currSeed = subject[0]
    currSubj = subject[1]

    if currSubj in skipSubjects:
        continue
    nrOfSeedsPerSubject[currSubj] += 1

    for aResults in subject[2]:
        for aResults2 in aResults:
            # If the np array comes from svm results
            if len(aResults2) < 6:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                if ares > 0.98:
                    ares = 0.99
                akernel = aResults2[2]
                
                
                # if "lin" in akernel:
                #     continue
                # if "sig" in akernel:
                #     continue
                aC = round(aResults2[3], 2)

                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [nameDict[aName], kernelDict[akernel], aC] not in combinationList:
                    combinationList.append([nameDict[aName], kernelDict[akernel], aC])
                    nrOfCombination += 1

                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aC]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                    
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])

                    # If the np array comes from NN results
            else:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                akernel = aResults2[2]
                aLz = round(aResults2[3], 2)
                areg = round(aResults2[4], 2)
                adrp = round(aResults2[5], 2)
                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [
                    nameDict[aName],
                    kernelDict[akernel],
                    aLz,
                    areg,
                    adrp,
                ] not in combinationList:
                    combinationList.append(
                        [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                    )
                    nrOfCombination += 1
                # combResult = []
                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                  
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])


# Making table

# headers = [f"{x[0]}:{x[1]}:{x[2]}:" for x in combinationList]
headers = np.arange(nrOfCombination)
subjectResultArray = np.empty([10, nrOfCombination], dtype=object)
subjectNrList = []

comboAverage = np.zeros([1, nrOfCombination])
comboTotal = np.zeros([1, nrOfCombination])
nrOfSubjects = np.zeros(nrOfCombination)

topTen = []
topThreeAllSubjects = []
nameArray = np.array(list(nameDict))
topPerFeature = np.zeros([10, len(nameDict)])
lowestTopRes = [0, 0]
for subNr, cResult in enumerate(combinationResult):
    topThreeThisSubject = []
    resArray = np.zeros([nrOfCombination])
    if len(cResult) != 0:
        for res in cResult:
            if onlySignificant == True:
                if res[1] < minAcc:
                    continue
            comboAverage[0, res[0]] = comboAverage[0, res[0]] + res[1]
            comboTotal[0, res[0]] = comboTotal[0, res[0]] + res[1]
            nrOfSubjects[res[0]] += 1
            subjectResultArray[subNr, res[0]] = res[1]
            
            comboName = list(nameDict.keys())[combinationList[res[0]][0]]
            nameIndex = np.where(nameArray == comboName)[0][0]
            if res[1] > topPerFeature[subNr, nameIndex]:
                topPerFeature[subNr, nameIndex] = res[1]

            if len(topTen) < 10 or topTen[0][0] < res[1]:

                if len(topTen) == 10:
                    topTen.reverse()
                    topTen.pop()

                dataF = list(nameDict.keys())[combinationList[res[0]][0]]
                kernel = list(kernelDict.keys())[combinationList[res[0]][1]]
                cVal = combinationList[res[0]][2]
                topTen.append([res[1], subNr, dataF, kernel, cVal])
                topTen.sort()

            
            
            if len(topThreeThisSubject) < 2 or topThreeThisSubject[0][0] < res[1]:

                if len(topThreeThisSubject) == 2:
                    topThreeThisSubject.reverse()
                    topThreeThisSubject.pop()

                dataF = list(nameDict.keys())[combinationList[res[0]][0]]
                kernel = list(kernelDict.keys())[combinationList[res[0]][1]]
                cVal = combinationList[res[0]][2]
                topThreeThisSubject.append([res[1], subNr, dataF, kernel, cVal])
                topThreeThisSubject.sort()
    if len(topThreeThisSubject)>0:
        topThreeAllSubjects.append(topThreeThisSubject)
# Print the table!
comboAverage = comboAverage / nrOfSubjects
print(comboAverage.shape)
print(subjectResultArray.shape)
pd.set_option("display.max_columns", None)

# For loop through zip of headers and columns,
# If column has all values == None, Remove column and header.
subjectResultArray = np.array(subjectResultArray)
emptyCols = []
atleastOneEmpty = False
for hcol, in zip(headers):
    if all(subjectResultArray[:,hcol] == None):
        atleastOneEmpty = True
        emptyCols.append(hcol)
if atleastOneEmpty: 
    subjectResultArray = np.delete(subjectResultArray, np.array(emptyCols),1)
    headers = np.delete(headers, np.array(emptyCols),0)
    comboAverage = np.delete(comboAverage, np.array(emptyCols),1)
    comboTotal = np.delete(comboTotal, np.array(emptyCols),1)
    nrOfSubjects = np.delete(nrOfSubjects, np.array(emptyCols),0)


table = pd.DataFrame(subjectResultArray, columns=headers)
display(table)




(1, 81)
(10, 81)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,0.225,0.215,0.1925,0.2475,0.24,0.2175,0.22,0.225,0.25,0.2625,0.235,0.265,0.53,0.5425,0.545,0.5725,0.54,0.59,0.34,0.3475,0.3825,0.675,0.645,0.6575,0.645,0.6225,0.6525,0.58,0.6,0.64,0.24,0.2475,0.235,0.645,0.5425,0.615,0.5075,0.5025,0.515,0.4975,0.55,0.54,0.22,0.265,0.2375,0.2425,0.275,0.2075,0.25,0.2525,0.2525,0.2475,0.2175,0.24,0.6825,0.6675,0.6625,0.6775,0.6075,0.7,0.6,0.6,0.6675,0.64,0.6425,0.635,0.615,0.6325,0.605,0.655,0.6725,0.6475,0.2325,0.2625,0.2325,0.57,0.6525,0.6275,0.5925,0.6075,0.615
2,0.25625,0.2375,0.2625,0.183333,0.185417,0.179167,0.254167,0.2875,0.233333,0.25625,0.270833,0.275,0.65625,0.672917,0.683333,0.616667,0.65625,0.620833,0.397917,0.3625,0.385417,0.7125,0.704167,0.735417,0.672917,0.639583,0.658333,0.647917,0.652083,0.685417,0.247917,0.254167,0.216667,0.65,0.652083,0.68125,0.575,0.6375,0.597917,0.60625,0.59375,0.602083,0.25625,0.229167,0.225,0.2875,0.26875,0.327083,0.28125,0.26875,0.24375,0.260417,0.25,0.258333,0.683333,0.64375,0.647917,0.69375,0.5875,0.645833,0.56875,0.510417,0.552083,0.7125,0.708333,0.69375,0.68125,0.702083,0.66875,0.69375,0.670833,0.675,0.260417,0.25,0.210417,0.65625,0.633333,0.658333,0.664583,0.683333,0.652083
3,0.254545,0.259091,0.320455,0.295455,0.261364,0.261364,0.240909,0.218182,0.256818,0.225,0.220455,0.197727,0.575,0.486364,0.536364,0.536364,0.425,0.484091,0.290909,0.297727,0.365909,0.563636,0.506818,0.531818,0.490909,0.502273,0.490909,0.495455,0.481818,0.477273,0.231818,0.238636,0.229545,0.590909,0.540909,0.581818,0.404545,0.436364,0.518182,0.431818,0.422727,0.45,0.209091,0.209091,0.265909,0.268182,0.268182,0.304545,0.231818,0.238636,0.263636,0.252273,0.225,0.229545,0.579545,0.452273,0.479545,0.656818,0.477273,0.625,0.470455,0.334091,0.377273,0.509091,0.477273,0.513636,0.55,0.552273,0.511364,0.509091,0.493182,0.513636,0.236364,0.261364,0.231818,0.5,0.504545,0.488636,0.486364,0.465909,0.438636
4,0.24375,0.26875,0.247917,0.214583,0.2625,0.272917,0.239583,0.227083,0.241667,0.225,0.229167,0.191667,0.658333,0.59375,0.677083,0.7,0.539583,0.672917,0.51875,0.370833,0.554167,0.741667,0.739583,0.71875,0.733333,0.74375,0.704167,0.65625,0.68125,0.639583,0.241667,0.252083,0.247917,0.69375,0.658333,0.675,0.552083,0.627083,0.608333,0.579167,0.654167,0.629167,0.233333,0.229167,0.2375,0.29375,0.329167,0.247917,0.241667,0.26875,0.227083,0.210417,0.208333,0.210417,0.675,0.679167,0.675,0.65,0.66875,0.6625,0.64375,0.491667,0.66875,0.702083,0.702083,0.7125,0.7125,0.727083,0.727083,0.645833,0.664583,0.66875,0.241667,0.25,0.254167,0.589583,0.677083,0.66875,0.591667,0.677083,0.633333
5,0.20625,0.258333,0.235417,0.252083,0.235417,0.25,0.21875,0.260417,0.2625,0.210417,0.164583,0.189583,0.695833,0.654167,0.691667,0.664583,0.666667,0.654167,0.56875,0.435417,0.583333,0.754167,0.766667,0.770833,0.689583,0.752083,0.745833,0.589583,0.63125,0.633333,0.20625,0.18125,0.195833,0.685417,0.654167,0.68125,0.583333,0.63125,0.670833,0.541667,0.635417,0.61875,0.225,0.258333,0.239583,0.289583,0.2625,0.26875,0.254167,0.266667,0.279167,0.23125,0.189583,0.177083,0.71875,0.616667,0.69375,0.702083,0.697917,0.71875,0.635417,0.575,0.66875,0.747917,0.754167,0.758333,0.710417,0.758333,0.739583,0.65625,0.685417,0.685417,0.19375,0.15,0.189583,0.616667,0.68125,0.645833,0.635417,0.677083,0.685417
6,0.231818,0.227273,0.231818,0.290909,0.240909,0.270455,0.229545,0.227273,0.186364,0.220455,0.229545,0.186364

In [184]:
x = 0

pd.set_option("display.max_columns", None)
print()
nameDictNrs = np.ones([1, len(nameDict)])
nameDictNrs[0,:] = np.arange(len(nameDict)) * 3
print(nameDictNrs.shape)
#table = pd.DataFrame(nameDictNrs , columns=nameDict.keys())
table = pd.DataFrame(topPerFeature, columns=nameDict.keys())
#display(table)
display(table)
# for names in nameDict.keys():
#     print(names, end="        ")
#     x +=1
#     #print(x)



(1, 27)


,-fftData,-welchData,-dataHR,-dataCorr1d,-dataFFTCV-BC,-dataWCV-BC,-dataHRCV-BC,-fftDataBC,-welchDataBC,-dataHRBC,-gaussianData,-dataGCVBC,-gaussianDataBC,-dataCorr1dBC,-fftDatacn3,-welchDatacn3,-dataHRcn3,-dataCorr1dcn3,-dataFFTCV-BCcn3,-dataWCV-BCcn3,-dataHRCV-BCcn3,-fftDatacn3BC,-welchDatacn3BC,-dataHRcn3BC,-gaussianDatacn3,-gaussianDatacn3BC,-dataCorr1dcn3BC
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.225000,0.247500,0.250000,0.265000,0.545000,0.590000,0.382500,0.675000,0.652500,0.640000,0.247500,0.645000,0.515000,0.550000,0.265000,0.275000,0.252500,0.247500,0.682500,0.700000,0.667500,0.642500,0.632500,0.672500,0.262500,0.652500,0.615000
2,0.262500,0.185417,0.287500,0.275000,0.683333,0.656250,0.397917,0.735417,0.672917,0.685417,0.254167,0.681250,0.637500,0.606250,0.256250,0.327083,0.281250,0.260417,0.683333,0.693750,0.568750,0.712500,0.702083,0.693750,0.260417,0.658333,0.683333
3,0.320455,0.295455,0.256818,0.225000,0.575000,0.536364,0.365909,0.563636,0.502273,0.495455,0.238636,0.590909,0.518182,0.450000,0.265909,0.304545,0.263636,0.252273,0.579545,0.656818,0.470455,0.513636,0.552273,0.513636,0.261364,0.504545,0.486364
4,0.268750,0.272917,0.241667,0.229167,0.677083,0.700000,0.554167,0.741667,0.743750,0.681250,0.252083,0.693750,0.627083,0.654167,0.237500,0.329167,0.268750,0.210417,0.679167,0.668750,0.668750,0.712500,0.727083,0.668750,0.254167,0.677083,0.677083
5,0.258333,0.252083,0.262500,0.210417,0.695833,0.666667,0.583333,0.770833,0.752083,0.633333,0.206250,0.685417,0.670833,0.635417,0.258333,0.289583,0.279167,0.231250,0.718750,0.718750,0.668750,0.758333,0.758333,0.685417,0.193750,0.681250,0.685417
6,0.231818,0.290909,0.229545,0.229545,0.672727,0.636364,0.381818,0.706818,0.734091,0.654545,0.275000,0.693182,0.643182,0.647727,0.281818,0.256818,0.213636,0.268182,0.690909,0.686364,0.322727,0.711364,0.722727,0.684091,0.293182,0.634091,0.656818
7,0.337500,0.214583,0.285417,0.289583,0.666667,0.681250,0.468750,0.741667,0.691667,0.618750,0.247917,0.670833,0.622917,0.647917,0.227083,0.308333,0.272917,0.279167,0.672917,0.702083,0.687500,0.745833,0.718750,0.666667,0.245833,0.641667,0.637500
8,0.267500,0.237500,0.247500,0.282500,0.685000,0.685000,0.637500,0.740000,0.732500,0.660000,0.285000,0.685000,0.630000,0.650000,0.237500,0.212500,0.275000,0.265000,0.680000,0.682500,0.687500,0.690000,0.682500,0.682500,0.297500,0.672500,0.670000
9,0.210417,0.289583,0.222917,0.266667,0.452083,0.475000,0.312500,0.714583,0.712500,0.583333,0.250000,0.608333,0.541667,0.575000,0.275000,0.341667,0.258333,0.277083,0.695833,0.616667,0.547917,0.727083,0.704167,0.656250,0.252083,0.660417,0.622917


In [185]:
import matplotlib.pyplot as plt
ordeTopAverage = np.array(np.argsort(comboAverage, axis=1)/(len(combinationList))*len(dataF),dtype=int)
print(ordeTopAverage)
print(len(combinationList))
print(len(dataF))
print(len(topPerFeature))
print(len(combinationList)/len(dataF))
#print(ordeTopAverage)

# #np.sort(topPerFeature, axis=1)
# #np.argsort(comboAverage, axis=1)
# #sortA = np.argsort(topPerFeature, axis = 1)
# if len(nameArray) < 70:
#     plt.figure(figsize=[30,10])
#     plt.boxplot(topPerFeature)
#     plt.legend(nameArray)
#     print("")
# else: 
#     plt.figure(figsize=[30,10])
#     fig, ax = plt.subplots(figsize=[30,10])
#     plt.margins(y=1.5)
#     ax.set_ylim([np.min(topPerFeature[1:,ordeTopAverage[0][:topAmount]])-0.3,1])
#     plt.boxplot(topPerFeature[1:,ordeTopAverage[0][:topAmount]])
#     plt.legend(nameArray[ordeTopAverage[0][-topAmount:]])
#     plt.title(f"Best {topAmount} feature combos by average accuracy over subjects")
#     print(nameArray[ordeTopAverage[0][-topAmount:]])
    


[[ 1  8  5  8 11  0  6  7 11  0  1  1  0  6  1  4  4  0  1  0 11  7  0  0
   7  8  8  7  7  7  3  2  3  9  5  2  9  6  9  6  2 12  5  6  4  6 12  2
   1  5  4 12  2  9  2  4 12  8 11 12 12 11  5  8 11  5  8  3 10  9  4  9
  10  4 10 10  3 10 10  3  3]]
81
13
10
6.230769230769231


In [186]:
print("Top ten highest accuracies")

topTen.reverse()
for top in topTen:
    print(
            f"Accuracy {round(top[0],2)} : subject {top[1]}, feature:{top[2]}, kernel{top[3]}, C:{top[4]}"
        )


Top ten highest accuracies
Accuracy 0.77 : subject 5, feature:-fftDataBC, kernelrbf, C:2.5
Accuracy 0.77 : subject 5, feature:-fftDataBC, kernelsigmoid, C:2.5
Accuracy 0.76 : subject 5, feature:-welchDatacn3BC, kernelsigmoid, C:2.5
Accuracy 0.76 : subject 5, feature:-fftDatacn3BC, kernelrbf, C:2.5
Accuracy 0.75 : subject 5, feature:-fftDataBC, kernellinear, C:2.5
Accuracy 0.75 : subject 5, feature:-fftDatacn3BC, kernelsigmoid, C:2.5
Accuracy 0.75 : subject 5, feature:-welchDataBC, kernelsigmoid, C:2.5
Accuracy 0.75 : subject 5, feature:-fftDatacn3BC, kernellinear, C:2.5
Accuracy 0.75 : subject 5, feature:-welchDataBC, kernelrbf, C:2.5
Accuracy 0.75 : subject 7, feature:-fftDatacn3BC, kernelrbf, C:2.5


In [187]:
# top for each feature

In [188]:
print("Top ten highest accuracies")
#del(topThreeThisSubject[0])
#len(topThreeAllSubjects[0])
for subTop in topThreeAllSubjects:
    subTop.reverse()
topThreeAllSubjects.sort(reverse=True)
arrayOfTopFeatureNames = []
for subTop in topThreeAllSubjects:
    print("")
    #subTop.reverse()
    for top in subTop[:1]:
        arrayOfTopFeatureNames.append(top[2])
        print(
                f"Accuracy {round(top[0],2)} : subject {top[1]}, feature:{top[2]}, kernel{top[3]}, C:{top[4]}"
            )
#print(arrayOfTopFeatureNames)
arrayOfTopFeatureNames = np.array(arrayOfTopFeatureNames, dtype=object)
#print(np.array(arrayOfTopFeatureNames, dtype=object).shape)
newArrayOfTopFeatureNames = []
for topNames in arrayOfTopFeatureNames:
    temp = topNames.replace("--","-")
    temp2 = temp.replace("-B","%")
    splitArray = np.array(temp2.split("-"),dtype=object)
    for ind, stri in enumerate(splitArray):
        splitArray[ind] = stri.replace("%","-B")
    newArrayOfTopFeatureNames.append(splitArray)
    
  
newArrayOfTopFeatureNames = np.array(newArrayOfTopFeatureNames, dtype=object) 
#saveTopName = "top2udrli"
if saveTopName is not None:
    #print(newArrayOfTopFeatureNames)
    np.save(saveTopName, newArrayOfTopFeatureNames)
    print("TopFeatureNameArraySaved")


Top ten highest accuracies

Accuracy 0.77 : subject 5, feature:-fftDataBC, kernelrbf, C:2.5

Accuracy 0.75 : subject 7, feature:-fftDatacn3BC, kernelrbf, C:2.5

Accuracy 0.74 : subject 4, feature:-welchDataBC, kernelsigmoid, C:2.5

Accuracy 0.74 : subject 8, feature:-fftDataBC, kernellinear, C:2.5

Accuracy 0.74 : subject 2, feature:-fftDataBC, kernelrbf, C:2.5

Accuracy 0.73 : subject 6, feature:-welchDataBC, kernelrbf, C:2.5

Accuracy 0.73 : subject 9, feature:-fftDatacn3BC, kernellinear, C:2.5

Accuracy 0.7 : subject 1, feature:-dataWCV-BCcn3, kernelrbf, C:2.5

Accuracy 0.66 : subject 3, feature:-dataWCV-BCcn3, kernellinear, C:2.5
TopFeatureNameArraySaved


In [189]:
subjAv = np.zeros(subjectResultArray.shape[0])
for s, subj in enumerate(subjectResultArray):
    subjAv[s] = 0
    if all(subj == None):
        continue
    subjAv[s] = np.mean(subj[np.where(subj != None)])
print(subjAv)
pd.set_option("display.max_columns", None)
table = pd.DataFrame(np.array(comboAverage), columns=headers)
display(table)

print(
    f"Max average accuracy {np.max(np.array(comboAverage[np.where(np.isnan(comboAverage)==False)]))}"
)
maxVal = np.max(np.array(comboAverage[np.where(np.isnan(comboAverage) == False)]))
maxInd = int(np.where(comboAverage == maxVal)[1][0])
# print(headers[maxInd])
# print(nrOfSubjects[maxInd])
# print(list(nameDict.keys())[combinationList[maxInd][0]])
# print(list(kernelDict.keys())[combinationList[maxInd][1]])
# print(f"C: {combinationList[maxInd][2]}")


[0.         0.46234568 0.49284979 0.39862514 0.50030864 0.50514403
 0.47704826 0.49912551 0.50308642 0.44346708]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80
0,0.242929,0.250337,0.249743,0.245892,0.232243,0.236035,0.237551,0.241486,0.236418,0.245652,0.236204,0.224251,0.608144,0.58096,0.614575,0.618081,0.551511,0.601768,0.42085,0.37178,0.452433,0.698847,0.683632,0.691886,0.660968,0.66939,0.674722,0.593817,0.613948,0.619162,0.242563,0.242563,0.228826,0.655017,0.613678,0.650833,0.538855,0.584718,0.57779,0.554116,0.596907,0.585425,0.233072,0.241397,0.234112,0.273274,0.269398,0.263607,0.251406,0.249268,0.251688,0.254588,0.229583,0.22439,0.655985,0.625118,0.652597,0.67064,0.615484,0.668994,0.558704,0.498754,0.551713,0.681978,0.677243,0.685093,0.668127,0.685737,0.671965,0.626705,0.652891,0.642083,0.234701,0.246061,0.231014,0.584491,0.638552,0.62024,0.597647,0.62896,0.614238


Max average accuracy 0.6988468013468014


In [190]:
#np.argpartition(comboAverage, -4)
#print(headers[comboAverage.argsort()[0][-5:][::-1]])
top5 = headers[comboAverage.argsort()[0][-5:][::-1]]
top5Vals = comboAverage[0][comboAverage.argsort()[0][-5:][::-1]]
for maxInd,res in zip(top5, top5Vals):
    print(list(nameDict.keys())[combinationList[maxInd][0]])
    print(list(kernelDict.keys())[combinationList[maxInd][1]])
    print(res)
    print()

-fftDataBC
linear
0.6988468013468014

-fftDataBC
rbf
0.6918855218855219

-welchDatacn3BC
sigmoid
0.6857365319865321

-fftDatacn3BC
rbf
0.6850925925925926

-fftDataBC
sigmoid
0.6836321548821549



In [191]:
#np.argpartition(comboAverage, -4)
#print(headers[comboAverage.argsort()[0][-5:][::-1]])
top5 = headers[comboTotal.argsort()[0][-5:][::-1]]
top5Vals = comboTotal[0][comboTotal.argsort()[0][-5:][::-1]] /9
for maxInd,res in zip(top5, top5Vals):
    print(list(nameDict.keys())[combinationList[maxInd][0]])
    print(list(kernelDict.keys())[combinationList[maxInd][1]])
    print(res)
    print()

-fftDataBC
linear
0.6988468013468014

-fftDataBC
rbf
0.6918855218855219

-welchDatacn3BC
sigmoid
0.6857365319865321

-fftDatacn3BC
rbf
0.6850925925925926

-fftDataBC
sigmoid
0.6836321548821549



In [192]:
table = pd.DataFrame(np.array(comboTotal), columns=headers)
display(table)

maxVal2 = np.max(np.array(comboTotal[np.where(np.isnan(comboTotal) == False)]))

for maxInd2 in np.where(comboTotal == maxVal2)[1]:

    # maxInd2 = int(np.where(comboTotal == maxVal2)[1])
    print(headers[maxInd2])
    print(nrOfSubjects[maxInd2])
    print(
        f"Max total significant accuracy then averaged {maxVal2/nrOfSubjects[maxInd2]} "
    )

    print(list(nameDict.keys())[combinationList[maxInd2][0]])
    print(list(kernelDict.keys())[combinationList[maxInd2][1]])
    print(f"C: {combinationList[maxInd2][2]} ")


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80
0,2.186364,2.25303,2.247689,2.21303,2.090189,2.124318,2.137955,2.173371,2.127765,2.210871,2.125833,2.018258,5.473295,5.228636,5.531174,5.562727,4.963598,5.415909,3.787652,3.346023,4.071894,6.289621,6.152689,6.22697,5.948712,6.024508,6.0725,5.344356,5.52553,5.572462,2.183068,2.183068,2.059432,5.895152,5.523106,5.8575,4.849697,5.262462,5.200114,4.987045,5.372159,5.268826,2.097652,2.172576,2.107008,2.45947,2.424583,2.372462,2.262652,2.243409,2.265189,2.291288,2.06625,2.019508,5.903864,5.626061,5.873371,6.035758,5.539356,6.020947,5.028333,4.488788,4.965417,6.137803,6.095189,6.165833,6.013144,6.171629,6.047689,5.640341,5.876023,5.77875,2.112311,2.214545,2.079129,5.260417,5.74697,5.582159,5.378826,5.660644,5.528144


21
9.0
Max total significant accuracy then averaged 0.6988468013468014 
-fftDataBC
linear
C: 2.5 


In [193]:
# maxAcc = np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])])
# print(f"Max single accuracy position {np.where(subjectResultArray == maxAcc)}")
# print(np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])]))
